In [2]:
import torch 
import torch.nn as nn 
import streamlit as st
import numpy as np 
import plotly.graph_objects as go 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt 

In [3]:
class LogisticRegression:


    def __init__(self ,lower_0, upper_0, sample_size_0, noise_0,lower_1, upper_1, sample_size_1, noise_1,w,b):

        # Parameters
        self.lower_0 = lower_0
        self.upper_0 = upper_0
        self.sample_size_0 = sample_size_0
        self.noise_0 = noise_0
        self.lower_1 = lower_1
        self.upper_1 = upper_1
        self.sample_size_1 = sample_size_1
        self.noise_1 = noise_1
        self.w = w
        self.b = b

        # made by attributes
        self.x0 = torch.linspace(lower_0, upper_0, sample_size_0) + torch.tensor([noise_0])
        self.x1 = torch.linspace(lower_1, upper_1, sample_size_1) - torch.tensor([noise_1])
        self.X = torch.cat((self.x0, self.x1), dim=0)
        self.y = torch.cat((torch.zeros(len(self.x0)), torch.ones(len(self.x1))), dim=0)

        # stand alone 
        self.inter_and_extrapolation = torch.linspace(-3,3,1000)
        self.possible_weights = torch.linspace(-5,25,100)
        self.possible_biases = torch.linspace(-5,5,100)       
        self.loss_fn = nn.BCEWithLogitsLoss()

        
        # made by stand alone
        self.weight_m , self.bias_m = torch.meshgrid(self.possible_weights,self.possible_biases,indexing='ij')
        self.weight_f = self.weight_m.flatten()
        self.bias_f   = self.bias_m.flatten()


#-----------------------------------------------------------------
# Math for loss_landscape, loss for classes & confusion matrix 

    def Loss(self):
        L = []

        for weight,bias in zip(self.weight_f,self.bias_f):
            z = (weight * self.X) + bias
            loss = self.loss_fn(z,self.y)
            L.append(loss)
        #------------------------
        L_f = torch.as_tensor(L)
        L_m = L_f.view(100,100)
        L_min = torch.argmin(L_f)
        min_index_w , min_index_b = np.unravel_index(L_min,(100,100))

        secret_weight = self.possible_weights[min_index_w]
        secret_bias   = self.possible_biases[min_index_b]

        return L_f,L_m,secret_weight,secret_bias

#-----------------------------------------------------------
    # Data points, sigmoid curve
    def generate_plot(self):
        scatter_class_0 = go.Scatter(
            x=self.x0,
            y=torch.zeros(len(self.x0)),
            mode='markers',
            marker=dict(color='purple'),
            name='class purple'
        )
        scatter_class_1 = go.Scatter(
            x=self.x1,
            y=torch.ones(len(self.x1)),
            mode='markers',
            marker=dict(color='orange'),
            name='class orange'
        )

        z = (self.w * self.inter_and_extrapolation) + self.b

        non_linear_line = go.Scatter(
            x= self.inter_and_extrapolation,
            y=torch.sigmoid(z),
            mode='lines',
            line={'color': 'rgb(27,158,119)'},
            name='model'
        )


        layout = go.Layout(
            xaxis=dict(
                # range=[-3.1, 3.1],
                title='X',
                zeroline=True,
                zerolinewidth=2,
                zerolinecolor='rgba(205, 200, 193, 0.7)'
            ),
            yaxis=dict(
                # range=[-0.5, 1.5],
                title='Y',
                zeroline=True,
                zerolinewidth=2,
                zerolinecolor='rgba(205, 200, 193, 0.7)'
            ),
            # height=500,
            # width=2600
        )
        figure = go.Figure(data=[scatter_class_0, scatter_class_1, non_linear_line], layout=layout)
        return figure
    
# -----------------------------------

    def loss_landscape(self,L_f,L_m,secret_weight,secret_bias):
        
        # landscape
            loss_landscape = go.Surface(
                    x = self.weight_m,
                    y = self.bias_m,
                    z = L_m,
                    name ='Loss function landscape'
                )

            # global
            Global_minima = go.Scatter3d(
                x = (secret_weight,),     
                y = (secret_bias,),
                z = (min(L_f),),
                mode = 'markers',
                marker = dict(color='yellow',size=10,symbol='diamond'),
                name = 'Global minima'
            )


            # ball
            z = (self.w * self.X ) +self.b   #forward pass
            loss = self.loss_fn(z,self.y)

            ball = go.Scatter3d(
                    x = (self.w,),
                    y = (self.b,),
                    z = (loss,),
                    mode = 'markers',
                    marker= dict(color='red'),
                    name = 'loss'
            )

            # layout 
            layout = go.Layout(
                 scene= dict(
                      xaxis = dict(range=[-5,15],title='weight'),
                      yaxis = dict(range=[-5,15],title='bias'),
                      zaxis = dict(range=[0,25],title ='loss')
                 )
            )


            figure = go.Figure(data = [loss_landscape,Global_minima,ball])
            
            return figure

In [4]:
w_val = 1.0
b_val = 0.8


data = LogisticRegression(lower_0 = -2,upper_0 = 0, sample_size_0 = 8,noise_0 = 0.4,
                                  lower_1 = 0, upper_1 = 2, sample_size_1 = 8, noise_1 = 0.4,w = w_val,b = b_val)

In [5]:
figure_1 = data.generate_plot()
figure_1

In [6]:
L_f,L_m,secret_weight,secret_bias = data.Loss()

In [7]:
data.loss_landscape(L_f,L_m,secret_weight,secret_bias)

In [12]:
torch.sigmoid(torch.tensor(2))


tensor(0.8808)